<a href="https://colab.research.google.com/github/JoyNjihia/AI_For_Software_Engineering_Assignment_Week_Two/blob/main/Week2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>